<h3>Kaggle Challenge</h3>
<h1>Expedia Hotel Recommendations</h1>
<hr style="height:2px;border:none;color:#333;background-color:#333;"/>
<b>Part II - Data Processing</b>

### Imports

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 7.0)

In [ ]:
expediaDF = pd.read_csv('data/train.csv.gz', 
                         nrows=1000000,
                        compression='gzip',
                        error_bad_lines=False)

In [2]:
testingDF = pd.read_csv('data/test.csv.gz', 
                        nrows=100000,
                        compression='gzip',
                        error_bad_lines=False)

In [3]:
missingValues = expediaDF.isnull().sum().sort_values(ascending=False)
print('**Missing values** ')
print(missingValues[missingValues > 0])

,id,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
0,0,2015-09-03 17:09:54,2,3,66,174,37449,5539.0567,1,1,0,3,2016-05-19,2016-05-23,2,0,1,12243,6,6,204,27
1,1,2015-09-24 17:38:35,2,3,66,174,37449,5873.2923,1,1,0,10,2016-05-12,2016-05-15,2,0,1,14474,7,6,204,1540
2,2,2015-06-07 15:53:02,2,3,66,142,17440,3975.9776,20,0,0,1,2015-07-26,2015-07-27,4,0,1,11353,1,2,50,699
3,3,2015-09-14 14:49:10,2,3,66,258,34156,1508.5975,28,0,1,10,2015-09-14,2015-09-16,2,0,1,8250,1,2,50,628
4,4,2015-07-17 09:32:04,2,3,66,467,36345,66.7913,50,0,0,0,2015-07-22,2015-07-23,2,0,1,11812,1,2,50,538
5,5,2015-07-21 11:58:45,2,3,66,311,48189,359.8521,51,0,0,0,2015-07-22,2015-07-24,4,0,2,11827,1,2,50,447
6,6,2015-07-29 07:58:39,2,3,66,311,48189,237.3465,51,0,0,10,2015-08-02,2015-08-03,2,0,1,8271,1,2,50,696
7,7,2015-08-01 20:13:15,2,3,66,348,24811,216.5785,51,0,0,10,2015-08-03,2015-08-04,2,0,1,8291,1,2,50,191
8,8,2015-11-07 12:29:09,2,3,66,311,48189,2337.6754,51,0,0,0,2015-12-30,2015-12-31,2,0,1,8250,1,2,50,628
9,9,2015-11-08 16:21:37,2,3,66,311,48189,2539.7995,51,0,0,10,2016-01-02,2016-01-03,2,0,1,9145,1,2,50,364


In [ ]:
print(13525001/37670293)